## 3 кейс

**В этом кейсе вы будете рассчитывать:**
* retention
* rolling retention
* lifetime
* churn rate
* mau
* wau
* dau

**Важно**

Перед началом решения задачи выполните следующую ячейку - в ней скачиваются нужные файлы

In [1]:
!wget https://gist.github.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv

!wget https://gist.github.com/Vs8th/aacb80595d1d6aaa2e31eb735f8bc644/raw/entries.csv

!wget https://gist.github.com/Vs8th/0e827e9a608117345dd6585ab81e8c86/raw/metrics.txt

--2025-09-18 07:38:49--  https://gist.github.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv
Resolving gist.github.com (gist.github.com)... 140.82.116.4
Connecting to gist.github.com (gist.github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv [following]
--2025-09-18 07:38:49--  https://gist.githubusercontent.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14918 (15K) [text/plain]
Saving to: ‘registrations.csv’

registrations.csv   100%[===================>]  14.57K  --.-KB/s    in 0s      

2025-09-18 07:38:49 (57.3 M

Файлами для работы являются `registrations.csv` и `entries.csv`. В них хранятся данные о регистрациях пользователей и входа на платформу соответственно.

In [2]:
import csv
from datetime import datetime, timedelta

def proccess_files(file1, file2):
  all_data = {}
  with open(file1, 'r') as reg_file:
      reader = csv.reader(reg_file, delimiter=';')
      next(reader)
      for row in reader:
          user_id = row[0]
          registration_date = datetime.strptime(row[1], '%Y-%m-%d')
          if user_id not in all_data:
            all_data[user_id] = {
                'user_id': user_id,
                'reg': registration_date,
                'entries': []
            }

  with open(file2, 'r') as entry_file:
      reader = csv.reader(entry_file, delimiter=';')
      next(reader)
      for row in reader:
          user_id = row[0]
          all_data[user_id]['entries'].append(datetime.strptime(row[1], '%Y-%m-%d'))
  return all_data



### **Посчитайте Retention 15 дня (в процентах) для пользователей, зарегистрированных в январе**

Cохраните результат в переменную `retention_15_day`

**Примечание:** результат округлите до 5 знаков после запятой

In [3]:

all_data = proccess_files('registrations.csv', 'entries.csv')

start_date = datetime.strptime('2021-01-01', '%Y-%m-%d')
end_date = datetime.strptime('2021-01-31', '%Y-%m-%d')
count_user = 0
ret_user = 0

for user_data in all_data.values():
  ret_15_date = user_data.get('reg') + timedelta(days=15)
  if start_date <= user_data.get('reg') <= end_date:
    count_user += 1
    if ret_15_date in user_data['entries']:
      ret_user += 1

retention_15_day = round(ret_user/count_user*100,5)

In [4]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
# Открываем файл с правильными ответами
with open('metrics.txt', 'r') as f:
    answers = f.read().split('\n')

correct_answer = float(answers[0])

try:
    assert retention_15_day == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Rolling-retention 30 дня (в процентах) для пользователей из той же когорты**

Сохраните результат в переменную `rolling_retention`

**Примечание:** результат округлите до 5 знаков после запятой

In [5]:
all_data = proccess_files('registrations.csv', 'entries.csv')

start_date = datetime.strptime('2021-01-01', '%Y-%m-%d')
end_date = datetime.strptime('2021-01-31', '%Y-%m-%d')
count_user = 0
ret_user = 0

for user_data in all_data.values():
  if start_date <= user_data.get('reg') <= end_date:
    ret_30_date = (user_data.get('reg') + timedelta(days=30))
    count_user += 1
    for entry_date in user_data['entries']:
      if entry_date >= ret_30_date:
        ret_user +=1
        break

rolling_retention = round(ret_user/count_user*100, 5)

In [6]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[1])

try:
    assert rolling_retention == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Lifetime по всем пользователям, посчитанный как интеграл от n-day retention**

Сохраните результат в переменную `lifetime`

**Примечание:** результат округлите до 5 знаков после запятой

In [7]:
from collections import defaultdict

all_data = proccess_files('registrations.csv', 'entries.csv')

users_by_day = defaultdict(set)
total_users = len(all_data)

for user_id, user_data in all_data.items():
  for entry_date in user_data['entries']:
    days_after_reg = (entry_date - user_data['reg']).days
    users_by_day[days_after_reg].add(user_id)

lifetime = 0
for day in sorted(users_by_day.keys()):
  lifetime += len(users_by_day[day])/total_users

lifetime = round(lifetime,5)

In [8]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[2])

try:
    assert lifetime == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Churn rate 29 дня (в долях), посчитанный по всем пользователям**

Сохраните результат в переменную `churn_29`.  
Если будете считать CR от Retention, ведите расчет от Rolling Retention, таким образом, мы получим всех, кто не заходил в 29 день и после. Ведя расчет от обычного Retention, наоборот - получим только CR в 29 день.


In [9]:
all_data = proccess_files('registrations.csv', 'entries.csv')

count_user = 0
ret_user = 0

for user_data in all_data.values():
  ret_29_date = (user_data.get('reg') + timedelta(days=29))
  count_user += 1
  for entry_date in user_data['entries']:
    if entry_date >= ret_29_date:
      ret_user +=1
      break

churn_29 = round(1-(ret_user/count_user), 5)

In [10]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[3])

try:
    assert churn_29 == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Mau, Wau, Dau за последний месяц/неделю/день записей**

Сохраните результат в переменные `dec_mau`, `dec_wau`, `dec_dau` соответственно

**Примечание:** последний месяц записей - декабрь. Поэтому `mau` рассчитываем для декабря (2021 года), для `wau` берем последнюю неделю - с 25 по 31 декабря, и для `dau` соответственно последний день - 31 декабря.

In [11]:
all_data = proccess_files('registrations.csv', 'entries.csv')

data_entries = [max(v['entries']) for v in all_data.values()]
max_date = max(data_entries)
month_delta = max_date - timedelta(days=30)
unique_users = set()

for user_data in all_data.values():
  for entry_date in user_data['entries']:
    if month_delta <= entry_date <= max_date:
      unique_users.add(user_data['user_id'])
dec_mau = len(unique_users)


In [12]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[4])

try:
    assert dec_mau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [13]:
all_data = proccess_files('registrations.csv', 'entries.csv')

data_entries = [max(v['entries']) for v in all_data.values()]
max_date = max(data_entries)
week_delta = max_date - timedelta(days=6)
unique_users = set()

for user_data in all_data.values():
  for entry_date in user_data['entries']:
    if week_delta <= entry_date <= max_date:
      unique_users.add(user_data['user_id'])

dec_wau = len(unique_users)

In [14]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[5])

try:
    assert dec_wau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [15]:
all_data = proccess_files('registrations.csv', 'entries.csv')

data_entries = [max(v['entries']) for v in all_data.values()]
max_date = max(data_entries)
unique_users = set()

for user_data in all_data.values():
  for entry_date in user_data['entries']:
    if entry_date == max_date:
      unique_users.add(user_data['user_id'])

dec_dau = len(unique_users)

In [16]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[6])

try:
    assert dec_dau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Mau, Wau, Dau усредненные**

Сохраните результат в переменные `avg_mau`, `avg_wau`, `avg_dau` соответственно

**Примечание:** результаты округлите до 5 знаков после запятой

In [17]:
all_data = proccess_files('registrations.csv', 'entries.csv')

users_by_month = defaultdict(set)

for user_id, user_data in all_data.items():
  for entry_date in user_data['entries']:
    if entry_date.month in list(range(1, 13)):
      users_by_month[entry_date.month].add(user_id)

avg_mau = 0
for month in sorted(users_by_month.keys()):
  avg_mau += len(users_by_month[month])/12

avg_mau = round(avg_mau, 5)

In [18]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[7])

try:
    assert avg_mau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [19]:
all_data = proccess_files('registrations.csv', 'entries.csv')

users_by_week = defaultdict(set)

for user_id, user_data in all_data.items():
  for entry_date in user_data['entries']:
    year, week, _ = entry_date.isocalendar()
    users_by_week[(year, week)].add(user_id)

week_users = [len(users) for users in users_by_week.values()]
avg_wau = round(sum(week_users)/len(week_users), 5)


In [20]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[8])

try:
    assert avg_wau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [21]:
all_data = proccess_files('registrations.csv', 'entries.csv')

users_by_days = defaultdict(set)

for user_id, user_data in all_data.items():
  for entry_date in user_data['entries']:
    year, month, day = entry_date.isocalendar()
    users_by_days[(year, month, day)].add(user_id)

day_users = [len(users) for users in users_by_days.values()]
avg_dau = round(sum(day_users)/len(day_users), 5)

In [22]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[9])

try:
    assert avg_dau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!
